# 第二次社会计算实验报告
author: [scarletborder](http://blog.scarletborders.top/)  
## [deprecated]获取所有榜单上电影的链接
**2023/12/22废弃，因为最火电影的逻辑改成了2023年度的最火10部电影。请用下面的新逻辑**   
豆瓣上只有2022年的榜单，所以用这个了。  
https://movie.douban.com/annual/2022/?fullscreen=1&source=navigation  
单个榜单的最高电影  
链接：".subject-top module-top10-grid-subject-top"  .child的第一个.subject-top-wrap的href  
评分".rating-card-value"  
其他上榜电影    
".subjects-rank-item" .child() 第一个.subjects-rank-subject的href  
评分".subjects-rank-rating"  

In [11]:
urlToRate:dict = dict()

In [5]:
from DrissionPage import WebPage

page = WebPage("s")
page.get(r"https://movie.douban.com/annual/2022/?fullscreen=1&source=navigation")
page.wait.ele_loaded(".container-content")
page.change_mode("d")
topFilms = page.eles(".subject-top module-top10-grid-subject-top")
for item in topFilms:
    url = item.child(".subject-top-wrap").attr("href")
    rate = item.ele(".rating-card-value").text
    urlToRate[url] = float(rate)

normalFilms = page.eles(".subjects-rank-item")
for item in normalFilms:
    url = item.child(".subjects-rank-subject").attr("href")
    rate = item.ele(".subjects-rank-rating").text
    urlToRate[url] = float(rate)

## 获取所有榜单上电影的链接[new]
定位到猫眼票房数据网中'https://piaofang.maoyan.com/rankings/year',选择`<li data-com="canTouch" data-loaded="true" class="">2023年</li>`。  
电影名称：`.first-line`，票房万元`.col2 tr`  
新的列表为list[tuple],tuple存放(电影名称，电影票房（万元）)。  
接着要找到豆瓣上的相应电影的资料以获取后续的信息。  
豆瓣电影门户'https://movie.douban.com/'  
搜索框`#inp-query`,搜索按键`tag:input@@type=submit@@value=搜索`

In [8]:
name2Money = list()

In [9]:
url = r"file:///res/boxhouse.html"
from DrissionPage import WebPage

page = WebPage("s")
page.get(url)
cols = page.eles('css:ul[class="row"][data-com]')
for col in cols[0:10]:
    name = col.ele(".first-line").text
    money = col.ele(".col2 tr").text
    name2Money.append((name, money))

In [12]:
urlList = list()

In [13]:
page = WebPage("d")

page.get(r"https://movie.douban.com/")
for film in name2Money:
    name = film[0]
    inputspan = page.ele("#inp-query")
    inputspan.input(name)
    page.ele("tag:input@@type=submit@@value=搜索").click()
    div = page.ele('#root')
    myurl = div.ele('css:div>div>div>div>div:nth-of-type(1)').ele('css:a[class="cover-link"]')
    urlList.append(myurl.attr('href'))

## 获取每个影片的数据
获取到的信息存储在文件，并命名为豆瓣`id_(title)`

### 基本信息
"#content"  
#### "@h1"
片名"tag:span@property=v:itemreviewed"  
年份"tag:span@class="year""  

#### "#info"
导演"tag:span@class=attrs" ，eles "@rel=v:directedBy"    
类型eles "tag:span@property="v:genre""     

#### "#link-report-intra"
剧情简介 "tag:span@property="v:summary""  

In [42]:
def query(title,url,money):
    # 获得电影的基本信息
    page = WebPage("d")
    page.get(url)
    page.wait.ele_loaded("#content")
    # page.change_mode("s")
    content = page.ele("#content")
    h1 = content.ele("tag:h1")
    # title = h1.ele(r'css:span[property="v\:itemreviewed"]').text
    year = h1.ele(r"tag:span@class=year").text

    info = content.ele("#info")
    directors_eles = info.ele(r"tag:span@class=attrs").eles(
        r'css:a[rel="v\:directedBy"]'
    )
    directors = list()
    for di in directors_eles:
        directors.append(di.text)

    file_type = info.ele(r'css:span[property="v\:genre"]').text

    intra = content.ele(r"#link-report-intra")
    brief = intra.ele(r'css:span[property="v\:summary"]').text

  
    with open(f"./data/{title}.md","a",encoding='utf-8') as f:
        f.write(f"""# {title}  
- 年份:{year}
- 导演:{directors}
- 类型:{file_type}
- 票房:{money}万元 
## 简述
{brief}\n""")



In [43]:
for idx in range(10):
    print(idx)
    query(name2Money[idx][0],urlList[idx],name2Money[idx][1])

0
1
2
3
4
5
6
7
8
9


## 情感分析
本次实验中采用4个角度分析情感。  
1. 好评，中评，差评个数。
   - 较为简单的方式但是能够总体概括一部影片的口碑
   - 长评(倍率1)：好5/4星，中3星,差2/1星
   - 短评(倍率0)：防止垃圾打分的短评污染数据所以不予考虑短评。

2. 最新评论中选取前`Cfg.ReviewCnt`个评论进行语义分析。
   - 由于用户偏好，热门评论可能更集中于某影片的显著优势评分的类上,采用最新评论的方式可能可以更客观地反应电影的总体评价。


## 粉丝属性
用户地理信息，网龄，观影数量。  
> 获取的信息均是通过情感分析获得的评论的用户的个人主页获取。  
> IP属地，注册日期，看过的电影数量。

In [23]:
for title,rate in name2Money:
    with open(f"./data/{title}.md", "a",encoding='utf-8') as f:
        f.write(
            f"## 评价比率\n"
        )

In [45]:
# 1. 各分类长评个数
def extractNum(s):
    s = s.split("(")[-1]
    s = s.split(")")[0]
    return int(s)


def Method1(url, title):
    LongReviewPage = f"{url}reviews"
    print(LongReviewPage)
    page = WebPage("d")
    page.get(LongReviewPage)
    page.wait.ele_loaded(r"css:ul[class='droplist']")
    dropdown = page.ele(r"css:ul[class='droplist']")
    # page.change_mode("s")

    rat5Num = dropdown.ele(r'css:a[href="\?rating\=5"]').text
    rat5Num = extractNum(rat5Num)
    rat4Num = dropdown.ele(r'css:a[href="\?rating\=4"]').text
    rat4Num = extractNum(rat4Num)
    rat3Num = dropdown.ele(r'css:a[href="\?rating\=3"]').text
    rat3Num = extractNum(rat3Num)
    rat2Num = dropdown.ele(r'css:a[href="\?rating\=2"]').text
    rat2Num = extractNum(rat2Num)
    rat1Num = dropdown.ele(r'css:a[href="\?rating\=1"]').text
    rat1Num = extractNum(rat1Num)

    sum = rat5Num + rat4Num + rat3Num + rat2Num + rat1Num
    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        good = "%.2f%%" % ((rat5Num+rat4Num)/sum * 100)
        med = "%.2f%%" % ((rat3Num)/sum * 100)
        bad = "%.2f%%" % ((rat2Num+rat1Num)/sum * 100)
        f.write(
            f"### Method1(好评，中评，差评个数)\n1. 好评:{good}\n2. 中评:{med}\n3. 差评:{bad}\n"
        )


for idx in range(10):
    Method1(urlList[idx], name2Money[idx][0])

https://movie.douban.com/subject/25845392/reviews
https://movie.douban.com/subject/26363254/reviews
https://movie.douban.com/subject/34841067/reviews
https://movie.douban.com/subject/26794435/reviews
https://movie.douban.com/subject/35267208/reviews
https://movie.douban.com/subject/35766491/reviews
https://movie.douban.com/subject/27619748/reviews
https://movie.douban.com/subject/26100958/reviews
https://movie.douban.com/subject/35613853/reviews
https://movie.douban.com/subject/35267208/reviews


### 获得评论信息
在上述过程中给每个电影创建了文本文件，并在其中存储了基本信息。  
为了得到语义信息，接下来每个影片网页中找到评论区

### 语义情感分析模块
[参考](https://blog.csdn.net/qq_39187675/article/details/85100560)  
[参考2](https://blog.csdn.net/LSGO_MYP/article/details/116949249)  
由于jupyter限制，爬取评论得到粉丝属性，并计算情感分数的部分请看`revAndanal.py`  

In [2]:
import pickle


class fileobj:
    def __init__(self, dd, ii, ff, ss) -> None:
        self.dd = dd  # 豆瓣账号创建日期列表的列表
        self.ii = ii  # 属地列表的列表
        self.ff = ff  # 观影次数列表的列表
        self.ss = ss  # 电影情感得分列表的列表


with open("./data/mydata.plk", "rb") as f:
    obj: fileobj = pickle.load(f)

In [47]:
# 通过方法2得到评价比率
## 由于汉语本身倾向的好评性
## 这里规定[-inf, 0]为差评,[0,10]为中评,[10,+inf]为好评
for idx in range(10):
    title = name2Money[idx][0]
    good = 0
    med = 0 
    bad = 0
    for score in obj.ss[idx]:
        if score < 0.0:
            bad += 1
        elif 0.0 <= score < 10.0:
            med += 1
        elif score >= 10.0:
            good += 1
    sum = good + med + bad
    good = "%.2f%%" % ((good)/sum * 100)
    med = "%.2f%%" % ((med)/sum * 100)
    bad = "%.2f%%" % ((bad)/sum * 100)
    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        f.write(f"""### Method2(最新评论的情感分析)
1. 好评:{good }
2. 中评:{med }
3. 差评:{bad }
""")


## 统计分析粉丝属性

In [27]:
for title, _ in name2Money:
    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        f.write(f"## 粉丝属性\n")

In [48]:
# 网龄
from datetime import datetime, timedelta
import numpy as np

for idx in range(10):
    title = name2Money[idx][0]
    ageList = list()
    for dateString in obj.dd[idx]:
        if len(dateString) < 4:
            continue
        datei = datetime.strptime(dateString, r"%Y-%m-%d")
        age = (datetime.now() - datei).days
        ageList.append(int(age))

    avg = np.mean(ageList)
    avg = int(avg)
    medium = np.median(ageList)
    medium = int(medium)
    counts = np.bincount(ageList)
    #返回众数
    bigpart = np.argmax(counts)
    bigpart = int(bigpart)

    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        f.write(
            f"""### 网龄分析
- 平均数{avg} days
- 中位数{medium} days
- 众数{bigpart} days
""")

In [49]:
# 观众ip属地

for idx in range(10):
    title = name2Money[idx][0]
    addrDict = dict()
    for addr in obj.ii[idx]:
        if addr == "" or addr == " ":
            continue
        item = addrDict.get(addr, 0)
        addrDict[addr] = item + 1
    que = sorted(addrDict.items(), key=lambda x: x[1], reverse=True)
    lan = [_[1] for _ in que]
    sum = np.sum(lan)
    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        f.write("### IP属地分析\n")
        lineCnt = 1
        for item in que:
            rate = item[1] / sum
            rate = "%.2f%%" % (rate * 100)
            f.write(f"{lineCnt}. {item[0]} {rate}\n")
            lineCnt += 1

In [50]:
# 观影数量
import numpy as np

for idx in range(10):
    title = name2Money[idx][0]
    filmNumList = list()
    for watchnum in obj.ff[idx]:
        if watchnum == -1 or watchnum == "" or watchnum == "-1":
            continue
        watchnum: str
        watchnum = watchnum.strip(")")
        watchnum = watchnum.strip("(")
        watchnum = watchnum.strip("（")
        watchnum = watchnum.strip("）")
        watchnum = watchnum.strip("看过")
        watchnum = watchnum.strip("部")
        filmNumList.append(int(watchnum))

    avg = np.mean(filmNumList)
    avg = int(avg)
    medium = np.median(filmNumList)
    m = int(medium)
    counts = np.bincount(filmNumList)
    # 返回众数
    bigpart = np.argmax(counts)
    bigpart = int(bigpart)

    with open(f"./data/{title}.md", "a", encoding="utf-8") as f:
        f.write(
            f"""### 观影数量分析
- 平均数{avg} 部
- 中位数{medium} 部
- 众数{bigpart} 部
""")